In [1]:
import torch
import os
import numpy as np
import random 

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 31
seed_everything(SEED)

import flair
flair.device = torch.device('cuda:0') 

In [2]:
from oracle import HybridOracle
from active_learner import ActiveLearner

In [3]:
EXPERIMENT = 'weighted_sampling'

In [4]:
oracle = HybridOracle(
    experiment_name=EXPERIMENT,
    all_data_file='../twitter_dataset/train_10.csv',
    valid_file='../twitter_dataset/valid.csv',
    test_file='../twitter_dataset/test.csv'
)

../twitter_dataset/train_10.csv
Starting with labelled data


In [5]:
sentences = oracle.get_all_sentences()

In [6]:
learner = ActiveLearner(sentences,
                        experiment_name=EXPERIMENT,
                        oracle=oracle, 
                        embeddings_storage_mode='gpu')

In [7]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.data import Dictionary, Sentence

In [8]:
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                    FlairEmbeddings('news-forward'),
                    FlairEmbeddings('news-backward'),
                   ]

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

label_dict = Dictionary()
label_dict.idx2item = [b'0', b'1']
label_dict.item2idx = {b'0': 0, b'1': 1}
# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [9]:
classifier = classifier.load('../twitter_dataset/0_1/best-model.pt')

2019-10-19 13:35:30,052 loading file ../twitter_dataset/0_1/best-model.pt


In [10]:
classifier, opt_state = learner.step(classifier, step_num=0, max_sample_size=100000, labelling_step_size=128, step_lr=1e-5)

STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DONE 12/128 LABELS
DONE 13/128 LABELS
DONE 14/128 LABELS
DONE 15/128 LABELS
DONE 16/128 LABELS
DONE 17/128 LABELS
DONE 18/128 LABELS
DONE 19/128 LABELS
DONE 20/128 LABELS
DONE 21/128 LABELS
DONE 22/128 LABELS
DONE 23/128 LABELS
DONE 24/128 LABELS
DONE 25/128 LABELS
DONE 26/128 LABELS
DONE 27/128 LABELS
DONE 28/128 LABELS
DONE 29/128 LABELS
DONE 30/128 LABELS
DONE 31/128 LABELS
DONE 32/128 LABELS
DONE 33/128 LABELS
DONE 34/128 LABELS
DONE 35/128 LABELS
DONE 36/128 LABELS
DONE 37/128 LABELS
DONE 38/128 LABELS
DONE 39/128 LABELS
DONE 40/128 LABELS
DONE 41/128 LABELS
DONE 42/128 LABELS
DONE 43/128 LABELS
DONE 44/128 LABELS
DONE 45/128 LABELS
DONE 46/128 LABELS
DONE 47/128 LABELS
DONE 48/128 LABELS
DONE 49/128 LABELS
DONE 50/128 LABELS
DONE 51/128 LABELS
DONE 52/128 LABELS
DO

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:38:02,032 ----------------------------------------------------------------------------------------------------
2019-10-19 13:38:02,034 Testing using best model ...
2019-10-19 13:38:02,036 loading file weighted_sampling/0/best-model.pt
2019-10-19 13:39:20,795 0.7015	0.7016	0.7015
2019-10-19 13:39:20,798 
MICRO_AVG: acc 0.5403 - f1-score 0.7015
MACRO_AVG: acc 0.54 - f1-score 0.70125
0          tp: 3675 - fp: 1669 - fn: 1315 - tn: 3342 - precision: 0.6877 - recall: 0.7365 - accuracy: 0.5519 - f1-score: 0.7113
1          tp: 3341 - fp: 1316 - fn: 1669 - tn: 3675 - precision: 0.7174 - recall: 0.6669 - accuracy: 0.5281 - f1-score: 0.6912
2019-10-19 13:39:20,800 ----------------------------------------------------------------------------------------------------


In [ ]:
for step_num in range(1, 100):
    classifier, opt_state = learner.step(classifier, step_num=step_num, optimizer_state=opt_state, max_sample_size=100000, labelling_step_size=128, step_lr=1e-5)

STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DONE 12/128 LABELS
DONE 13/128 LABELS
DONE 14/128 LABELS
DONE 15/128 LABELS
DONE 16/128 LABELS
DONE 17/128 LABELS
DONE 18/128 LABELS
DONE 19/128 LABELS
DONE 20/128 LABELS
DONE 21/128 LABELS
DONE 22/128 LABELS
DONE 23/128 LABELS
DONE 24/128 LABELS
DONE 25/128 LABELS
DONE 26/128 LABELS
DONE 27/128 LABELS
DONE 28/128 LABELS
DONE 29/128 LABELS
DONE 30/128 LABELS
DONE 31/128 LABELS
DONE 32/128 LABELS
DONE 33/128 LABELS
DONE 34/128 LABELS
DONE 35/128 LABELS
DONE 36/128 LABELS
DONE 37/128 LABELS
DONE 38/128 LABELS
DONE 39/128 LABELS
DONE 40/128 LABELS
DONE 41/128 LABELS
DONE 42/128 LABELS
DONE 43/128 LABELS
DONE 44/128 LABELS
DONE 45/128 LABELS
DONE 46/128 LABELS
DONE 47/128 LABELS
DONE 48/128 LABELS
DONE 49/128 LABELS
DONE 50/128 LABELS
DONE 51/128 LABELS
DONE 52/128 LABELS
DO

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:41:57,104 ----------------------------------------------------------------------------------------------------
2019-10-19 13:41:57,105 Testing using best model ...
2019-10-19 13:41:57,107 loading file weighted_sampling/1/best-model.pt
2019-10-19 13:43:08,716 0.6921	0.6922	0.6921
2019-10-19 13:43:08,718 
MICRO_AVG: acc 0.5292 - f1-score 0.6921
MACRO_AVG: acc 0.5273 - f1-score 0.69
0          tp: 3880 - fp: 1968 - fn: 1110 - tn: 3043 - precision: 0.6635 - recall: 0.7776 - accuracy: 0.5576 - f1-score: 0.7160
1          tp: 3042 - fp: 1111 - fn: 1968 - tn: 3880 - precision: 0.7325 - recall: 0.6072 - accuracy: 0.4970 - f1-score: 0.6640
2019-10-19 13:43:08,719 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DON

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:45:41,478 ----------------------------------------------------------------------------------------------------
2019-10-19 13:45:41,480 Testing using best model ...
2019-10-19 13:45:41,482 loading file weighted_sampling/2/best-model.pt
2019-10-19 13:47:14,530 0.6815	0.6816	0.6815
2019-10-19 13:47:14,532 
MICRO_AVG: acc 0.517 - f1-score 0.6815
MACRO_AVG: acc 0.5118 - f1-score 0.67555
0          tp: 4088 - fp: 2282 - fn: 902 - tn: 2729 - precision: 0.6418 - recall: 0.8192 - accuracy: 0.5622 - f1-score: 0.7197
1          tp: 2728 - fp: 903 - fn: 2282 - tn: 4088 - precision: 0.7513 - recall: 0.5445 - accuracy: 0.4614 - f1-score: 0.6314
2019-10-19 13:47:14,533 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DON

...

2019-10-19 14:43:11,898 ----------------------------------------------------------------------------------------------------
2019-10-19 14:43:11,899 Testing using best model ...
2019-10-19 14:43:11,900 loading file weighted_sampling/16/best-model.pt
2019-10-19 14:44:21,774 0.5182	0.5183	0.5182
2019-10-19 14:44:21,776 
MICRO_AVG: acc 0.3498 - f1-score 0.5182
MACRO_AVG: acc 0.2754 - f1-score 0.378
0          tp: 4966 - fp: 4793 - fn: 24 - tn: 218 - precision: 0.5089 - recall: 0.9952 - accuracy: 0.5076 - f1-score: 0.6734
1          tp: 217 - fp: 25 - fn: 4793 - tn: 4966 - precision: 0.8967 - recall: 0.0433 - accuracy: 0.0431 - f1-score: 0.0826
2019-10-19 14:44:21,777 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DONE 12

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 14:46:43,903 ----------------------------------------------------------------------------------------------------
2019-10-19 14:46:43,905 Testing using best model ...
2019-10-19 14:46:43,906 loading file weighted_sampling/17/best-model.pt
2019-10-19 14:48:01,055 0.5153	0.5154	0.5153
2019-10-19 14:48:01,057 
MICRO_AVG: acc 0.3471 - f1-score 0.5153
MACRO_AVG: acc 0.2714 - f1-score 0.37135
0          tp: 4970 - fp: 4826 - fn: 20 - tn: 185 - precision: 0.5073 - recall: 0.9960 - accuracy: 0.5063 - f1-score: 0.6722
1          tp: 184 - fp: 21 - fn: 4826 - tn: 4970 - precision: 0.8976 - recall: 0.0367 - accuracy: 0.0366 - f1-score: 0.0705
2019-10-19 14:48:01,057 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/128 LABELS
DONE 2/128 LABELS
DONE 3/128 LABELS
DONE 4/128 LABELS
DONE 5/128 LABELS
DONE 6/128 LABELS
DONE 7/128 LABELS
DONE 8/128 LABELS
DONE 9/128 LABELS
DONE 10/128 LABELS
DONE 11/128 LABELS
DONE 

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 14:50:17,555 ----------------------------------------------------------------------------------------------------
2019-10-19 14:50:17,556 Testing using best model ...
2019-10-19 14:50:17,557 loading file weighted_sampling/18/best-model.pt
